In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
# https://colab.research.google.com/github/google/automl/blob/master/efficientnetv2/tfhub.ipynb download code
!pip install tensorflow_addons

import os
import sys
import tensorflow.compat.v1 as tf

if "efficientnetv2" not in os.getcwd():
  !git clone --depth 1 https://github.com/google/automl
  os.chdir('automl/efficientnetv2')
  sys.path.append('.')
else:
  !git pull

# download checkpoints ckpt
def download(m):
  if m not in os.listdir():
    !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/v2/{m}.tgz
    !tar zxf {m}.tgz
  ckpt_path = os.path.join(os.getcwd(), m)
  return ckpt_path

# https://colab.research.google.com/github/google/automl/blob/master/efficientnetv2/tfhub.ipynb
MODEL = 'efficientnetv2-b0'
import effnetv2_model

ckpt_path = download(MODEL)
if tf.io.gfile.isdir(ckpt_path):
  ckpt_path = tf.train.latest_checkpoint(ckpt_path)

In [ ]:
#import keras_efficientnet_v2
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

In [ ]:
data_dir = "/content/drive/MyDrive/food"

batch_size = 128
imgSize = 224

splitRate = 0.2

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir, seed=123, subset="training", validation_split=splitRate,
  image_size=(imgSize, imgSize), batch_size=batch_size
)

valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir, seed=123, subset="validation", validation_split=splitRate,
  image_size=(imgSize, imgSize), batch_size=batch_size
)

classNum = len(train_ds.class_names)
print(train_ds.class_names)

In [ ]:
data_augmentation = keras.Sequential(
  [
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1)
  ]
)

resize_and_rescale = tf.keras.Sequential([
  tf.keras.layers.Resizing(imgSize, imgSize),
  tf.keras.layers.Rescaling(1./127.5, offset=-1)
])

In [ ]:
expendRound = 3
temp_ds = train_ds
for i in range(expendRound):
    train_ds = train_ds.concatenate(temp_ds)

train_ds = train_ds.map(lambda image,label:(data_augmentation(image),label))
train_ds = train_ds.map(lambda image,label:(resize_and_rescale(image),label))

valid_ds = valid_ds.map(lambda image,label:(resize_and_rescale(image),label))

In [ ]:
# If you found that baseModel occur error, try to reset all cells and re-run
baseModel = effnetv2_model.get_model(MODEL, include_top=False)

baseModel.trainable = True
print("Layers count", len(baseModel.layers))

fine_tune_at = int( len(baseModel.layers) * 0.2) # 20
for layer in baseModel.layers[:fine_tune_at]:
  layer.trainable = False

model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=[imgSize, imgSize, 3]),
  baseModel,
  tf.keras.layers.Dropout(rate=0.2),
  tf.keras.layers.Dense(classNum, activation='softmax'),
])
  
epochsRound = 9
base_learning_rate = 0.0001

checkpoint_filepath = './tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_ds, epochs=epochsRound, validation_data=valid_ds, callbacks=[model_checkpoint_callback] )

now = datetime.now()
current_time = now.strftime("%b-%d-%Y_%H:%M:%S")
srt = "/content/drive/MyDrive/savedModel/" + "efficientnetV1B0" + current_time + ".h5"

model.load_weights(checkpoint_filepath)

test_loss, test_acc = model.evaluate(valid_ds, verbose=2)
print(test_acc)

model.save(srt)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochsRound)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# References  

1. EfficientNetV2:  
https://github.com/google/automl/tree/master/efficientnetv2  
2. Transfer learning and fine-tuning  
https://www.tensorflow.org/tutorials/images/transfer_learning?hl=zh-tw  
